# Patch Based vs Octree Based AMR

In [1]:
using Plots
using Symbolics

In [2]:
@variables r

1-element Vector{Num}:
 r

#### Number of Grid Points

In [3]:
# patch based AMR
GridPointsPatch = [
    256^3,  # 8192.0
    256^3,  # 4096.0
    256^3,  # 2048.0
    256^3,  # 1024.0
    256^3,  # 512.0
    256^3,  # 256.0
    256^3,  # 128.0
    256^3,  # 64.0
    256^3,  # 32.0
    (256 * 20/16)^3,  # 20.0
    (256 * 6/8)^3 * 2,  # 6.0
    (256 * 3/4)^3 * 2,  # 3.0
    (256 * 0.5/2)^3 * 2,  # 0.5
];
# octree based AMR
GridPointsOctree = [
    256^3           - (256 * 1/2)^3,  # 8192.0
    256^3           - (256 * 1/2)^3,  # 4096.0
    256^3           - (256 * 1/2)^3,  # 2048.0
    256^3           - (256 * 1/2)^3,  # 1024.0
    256^3           - (256 * 1/2)^3,  # 512.0
    256^3           - (256 * 1/2)^3,  # 256.0
    256^3           - (256 * 1/2)^3,  # 128.0
    256^3           - (256 * 1/2)^3,  # 64.0
    256^3           - (256 * 1/2)^3,  # 32.0
    (256 * 20/16)^3 - (((256 * 20/16) * 6/20)^3) * 2,  # 20.0
    ((256 * 6/8)^3  - ((256 * 6/8) *   3/6)^3) * 2,  # 6.0
    ((256 * 3/4)^3  - ((256 * 3/4) * 0.5/3)^3) * 2,  # 3.0
    (256 * 0.5/2)^3 * 2,  # 0.5
];

In [4]:
Levels = length(GridPointsPatch)

13

In [5]:
println(GridPointsPatch)
println(GridPointsOctree)

[1.6777216e7, 1.6777216e7, 1.6777216e7, 1.6777216e7, 1.6777216e7, 1.6777216e7, 1.6777216e7, 1.6777216e7, 1.6777216e7, 3.2768e7, 1.4155776e7, 1.4155776e7, 524288.0]
[1.4680064e7, 1.4680064e7, 1.4680064e7, 1.4680064e7, 1.4680064e7, 1.4680064e7, 1.4680064e7, 1.4680064e7, 1.4680064e7, 3.0998528e7, 1.2386304e7, 1.409024e7, 524288.0]


## Patch Based AMR

$$
\begin{align}
\sum_{l=1}^{L}2^{l-1}N_l
\end{align}
$$

In [6]:
patchZCs = sum(GridPointsPatch[lev] * 2^(lev-1) for lev in 1:Levels)

7.0984400896e10

## Octree Based AMR

$$
\begin{align}
2^{L-1}\sum_{l=1}^LN_l
\end{align}
$$

In [7]:
octreeZCs = 2^(Levels-1) * sum(GridPointsOctree[lev] for lev in 1:Levels)

7.78731257856e11

## Compare: the total number of ZCs required to make a full time step

In [8]:
octbypatZCs = octreeZCs / patchZCs

10.970456156936894

## Compare: the number of ZCs per node required to make a full time step

In [9]:
patchUs = 256^3 / r

16777216 / r

In [10]:
octreeUs = (octreeZCs/2^(Levels-1)) / r

1.90119936e8 / r

In [11]:
patchZCperU = patchZCs / patchUs

(7.0984400896e10r) / 16777216

In [12]:
octreeZCperU = octreeZCs / octreeUs

(7.78731257856e11r) / 1.90119936e8

#### compare

In [13]:
octbypatZCperU = simplify(octreeZCperU / patchZCperU)

0.9680926494918459

In [14]:
octbypatUs = simplify(octreeUs / patchUs)

11.33203125